In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
def sigmoid(x):
 return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
 return x * (1 - x)
def softmax(x):
 exp_vals = np.exp(x - np.max(x, axis=1, keepdims=True)) # stability trick
 return exp_vals / np.sum(exp_vals, axis=1, keepdims=True)

In [5]:
class MLPClassifier:
 def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
  self.lr = learning_rate
  self.W1 = np.random.randn(input_size, hidden_size) * 0.01
  self.b1 = np.zeros((1, hidden_size))
  self.W2 = np.random.randn(hidden_size, output_size) * 0.01
  self.b2 = np.zeros((1, output_size))
 def forward(self, X):
  self.z1 = np.dot(X, self.W1) + self.b1
  self.a1 = sigmoid(self.z1)
  self.z2 = np.dot(self.a1, self.W2) + self.b2
  self.a2 = softmax(self.z2)
  return self.a2
 def backward(self, X, y, output):
  m = X.shape[0]
  d_output = (output - y) / m
  d_hidden = np.dot(d_output, self.W2.T) * sigmoid_derivative(self.a1)
  # Update weights
  self.W2 -= self.lr * np.dot(self.a1.T, d_output)
  self.b2 -= self.lr * np.sum(d_output, axis=0, keepdims=True)
  self.W1 -= self.lr * np.dot(X.T, d_hidden)
  self.b1 -= self.lr * np.sum(d_hidden, axis=0, keepdims=True)
 def train(self, X, y, epochs=1000):
  for epoch in range(epochs):
   output = self.forward(X)
   self.backward(X, y, output)
   if epoch % 100 == 0:
    loss = -np.mean(np.sum(y * np.log(output + 1e-9), axis=1))
    print(f"Epoch {epoch}, Loss: {loss:.6f}")
 def predict(self, X):
  probs = self.forward(X)
  return np.argmax(probs, axis=1)

In [6]:
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)
class_names = iris.target_names # ['setosa', 'versicolor', 'virginica']

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [8]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
 X, y_encoded, test_size=0.2, random_state=42
)


In [10]:
mlp = MLPClassifier(input_size=4, hidden_size=8, output_size=3, learning_rate=0.1)
mlp.train(X_train, y_train, epochs=1000)

Epoch 0, Loss: 1.098504
Epoch 100, Loss: 1.087663
Epoch 200, Loss: 0.811359
Epoch 300, Loss: 0.532199
Epoch 400, Loss: 0.426292
Epoch 500, Loss: 0.360178
Epoch 600, Loss: 0.310714
Epoch 700, Loss: 0.271635
Epoch 800, Loss: 0.239613
Epoch 900, Loss: 0.212839


In [11]:
y_pred = mlp.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)
print("\nTest Accuracy:", accuracy)
print("True Labels: ", y_true)
print("Predicted Labels:", y_pred)


Test Accuracy: 0.9666666666666667
True Labels:  [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Predicted Labels: [1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [12]:
print("\nEnter flower measurements to predict Iris class:")
sepal_length = float(input("Sepal Length (cm): "))
sepal_width = float(input("Sepal Width (cm): "))
petal_length = float(input("Petal Length (cm): "))
petal_width = float(input("Petal Width (cm): "))
user_input = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
user_input_scaled = scaler.transform(user_input)
user_pred = mlp.predict(user_input_scaled)[0]
print("\nPredicted Class Label:", user_pred)
print("Predicted Class Name:", class_names[user_pred])


Enter flower measurements to predict Iris class:
Sepal Length (cm): 4
Sepal Width (cm): 5
Petal Length (cm): 6
Petal Width (cm): 7

Predicted Class Label: 2
Predicted Class Name: virginica
